<h1>Steel Industry Data - Logistic Regression Classification</h1>
<h2>IT 7143</h2>
<h3></h3>
<h3></h3>

In [1]:
#import pandas and numpy library and import the csv file
import pandas as pd
import numpy as np

data=pd.read_csv('Steel_industry_data.csv')
data.drop(columns=['date'],inplace=True)
#rename attributes
data = data.rename(columns = {'Lagging_Current_Reactive.Power_kVarh' : 'Lagging_Current_Reactive_Power_kVarh', 
                          'CO2(tCO2)' : 'CO2'})
data.head()
feature_cols=data.columns.drop('Load_Type')
print(data.shape)

(35040, 10)


In [2]:
#replace letters in data with numerical values
data.dropna(inplace=True)
data['WeekStatus']=data['WeekStatus'].map({'Weekday': 0, 'Weekend': 1})
data['Day_of_week']=data['Day_of_week'].map({'Monday': 0, 'Tuesday': 1,'Wednesday':2,'Thursday':3,"Friday":4,"Saturday":5,"Sunday":6})

In [3]:
#use feature columns as X and target load type as y
X = data[feature_cols]
y = data.Load_Type

In [4]:
data.head()

,Usage_kWh,Lagging_Current_Reactive_Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2,Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
0,3.17,2.95,0.0,0.0,73.21,100.0,900,0,0,Light_Load
1,4.00,4.46,0.0,0.0,66.77,100.0,1800,0,0,Light_Load
2,3.24,3.28,0.0,0.0,70.28,100.0,2700,0,0,Light_Load
3,3.31,3.56,0.0,0.0,68.09,100.0,3600,0,0,Light_Load
4,3.82,4.50,0.0,0.0,64.72,100.0,4500,0,0,Light_Load


In [5]:
#import random split library
from sklearn.model_selection import ShuffleSplit

#split the data once with 25% data as testing and a random seed of 30
split = ShuffleSplit(n_splits=1, test_size=0.25, random_state=30)

#use the for loop to obtain train_set and test_set as new dataframes
for train_index, test_index in split.split(data):
    train_set = data.loc[train_index]
    test_set = data.loc[test_index]
    
#check size of train_set and test_set
print(train_set.shape, test_set.shape)

(26280, 10) (8760, 10)


In [6]:
#create trainX with input data and trainY for target column
trainX = train_set.loc[:, :'Day_of_week']
trainY = train_set.loc[:, 'Load_Type']
trainX.shape, trainY.shape

((26280, 9), (26280,))

In [7]:
#same thing for test data
testX = test_set.loc[:,:'Day_of_week']
testY = test_set.loc[:,'Load_Type']

In [8]:
#use a pipeline for preprocessing
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np

num_cols = trainX.columns[(trainX.dtypes==np.int64) | (trainX.dtypes==np.float64)]

num_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='median')),
    ('standardize', StandardScaler())
])

cat_cols = trainX.columns[trainX.dtypes==object]

cat_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='constant',fill_value='missing')),
    ('encode', OneHotEncoder())
])

full_pipeline = ColumnTransformer([
    ('numeric', num_pipeline, num_cols),
    ('class', cat_pipeline, cat_cols)
])

In [9]:
trainX_prc = full_pipeline.fit_transform(trainX)

traindata = np.concatenate([trainY.values.reshape(-1,1),trainX_prc],axis=1)



testX_prc = full_pipeline.transform(testX)

testdata = np.array(testX_prc)

In [10]:
trainX_prc = full_pipeline.fit_transform(trainX)

trainX_prc.shape

(26280, 9)

In [11]:
testX_prc = full_pipeline.transform(testX)

testX_prc.shape

(8760, 9)

In [12]:
#import libraries for logistic regression model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score

#create new model
logistic = LogisticRegression()

#train 
logistic.fit(trainX_prc, trainY)

#get training accuracy
train_accuracy_lg = logistic.score(trainX_prc, trainY)

#get testing accuracy
test_accuracy_lg = logistic.score(testX_prc, testY)

print('Training Accuracy:', train_accuracy_lg)
print('Testing Accuracy:', test_accuracy_lg)

Training Accuracy: 0.7580289193302892
Testing Accuracy: 0.7478310502283105
